# Probability and Counting

In [1]:
import numpy as np
from scipy import stats, special

# Sampling and simulation

The function [`numpy.random.choice`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.choice.html) is a useful way of drawing random samples in NumPy. (Technically, they are pseudo-random since there is an underlying deterministic algorithm, but they "look like" random samples for almost all practical purposes.) For example,

In [2]:
# seed the random number generator
np.random.seed(1)

n = 10
k = 5
np.random.choice(np.arange(1, n+1), k, replace=False)

array([ 3, 10,  7,  5,  1])

generates a random sample of 5 of the numbers from 1 to 10, without replacement, and with equal probabilities given to each number. To sample with replacement instead, you can explicitly specify `replace=True`, or you may leave that argument out altogether since the default for `numpy.random.choice` is `replace=True`.

In [3]:
np.random.seed(1)

# Example: sampling with replacement
np.random.choice(np.arange(1, n + 1), k, replace=True)

array([ 6,  9, 10,  6,  1])

To obtain a random permutation of an `array` of numbers $1, 2, \ldots, n$ we can use [`numpy.random.shuffle`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.shuffle.html). Note that this function operates on the given `array` in-place.

In [4]:
np.random.seed(2)

m = 1
n = 10

v = np.arange(m, n + 1)
print('v =', v)

np.random.shuffle(v)
print('v, shuffled =', v)

v = [ 1  2  3  4  5  6  7  8  9 10]
v, shuffled = [ 5  2  6  1  8  3  4  7 10  9]


`numpy.random.choice` also allows us to specify general probabilities for sampling each number. For example,

In [5]:
np.random.seed(3)

# from the 4 numbers starting from 0
# obtain a sample of size 3
# with replacement
# using the probabilities listed in p
np.random.choice(4, 3, replace=True, p=[0.1, 0.2, 0.3, 0.4])

array([2, 3, 1])

samples 3 numbers between 0 and 3, with replacement, and with probabilities given by the parameter `p=[0.1, 0.2, 0.3, 0.4]`. If the sampling is without replacement, then at each stage the probability of any not-yet-chosen number is _proportional_ to its original probability.

# Matching problem simulation

Let's show by simulation that the probability of a matching card in Example 1.6.4 is approximately $1 − 1/e$ when the deck is sufficiently large. Using [`numpy.random.permutation`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.permutation.html#numpy.random.permutation) while iterating in a for-loop (see Python flow controls [`for`](https://docs.python.org/3/tutorial/controlflow.html#for-statements) and [`range`](https://docs.python.org/3/tutorial/controlflow.html#the-range-function)), we can perform the experiment a bunch of times and see how many times we encounter at least one matching card:

In [6]:
np.random.seed(8)

n = 100
trials = 10000

ordered = np.arange(1, n + 1)

results = np.empty(trials)

for trial in range(trials):
    shuffled = np.random.permutation(ordered)
    results[trial] = np.sum(shuffled == ordered)

ans = np.sum(results >= 1) / trials

expected = 1 - 1 / np.e

print(f'simulated value: {ans:.2F}')
print(f'expected value : {expected:.2F}')

simulated value: 0.62
expected value : 0.63


First, we declare and assign values to variables for the size of the deck `n`, and the number of `trials` in our simulation.

Next, we generate a sequence from 1 to `n` (stopping at `n+1` to include `n`) to represent our ordered deck of cards.

The code then loops for `trial` number of times, where
* a permutation of a new sequence from 1 to `n` is created
* the number of cards (indices) that match with our `ordered` sequence are counted as `m`
* the number of matches `m` are saved to a temporary accumulator array `tmp`

After completing `trial` simulations, we create a NumPy `array` `results` from the `tmp` accumulator, which lets us count the number of simulations where there was at least 1 match.

Finally, we add up the number of times where there was at least one matching card, and we divide by the number of simulations.

## Birthday problem calculation and simulation

The following code uses the NumPy module functions [`numpy.prod`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.prod.html)  (which gives the product of the elements of the given `array`) and [`numpy.sum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html) to calculate the probability of at least one birthday match in a group of 23 people:

In [7]:
k = 23
days_in_year = 365

numer = np.arange(days_in_year, days_in_year - k, -1)
denom = np.array([days_in_year] * k)

1.0 - np.sum(np.prod(numer / denom))

0.5072972343239857

Unfortunately, NumPy and SciPy do not have library functions such as [`pbirthday` and `qbirthday`](https://www.rdocumentation.org/packages/stats/versions/3.5.1/topics/birthday) for computing birthday probabilities as does R. However, you can easily compose your own functions.

In [8]:
def pbirthday_naive(n):
    """ Return the probability of at least 1 matching pair of birthdays out of n people.
        Assumes 365-day year.
    
        This is a naive implementation that may overflow or error under certain input.
    """
    days_in_year = 365
    if n < 2:
        return 0.0
    elif n > days_in_year:
        return 1.0
    else:
        numer = np.arange(days_in_year, days_in_year - n, -1)
        denom = np.array([days_in_year] * n)
        return 1.0 - np.sum(np.prod(numer / denom))


def qbirthday_naive(p):
    """ Return an approximation of the number of people required to have at least 1 birthday
        match with probability p.
    """
    raw = np.sqrt(2 * 365 * np.log(1.0 / (1.0 - p)))
    return np.ceil(raw).astype(np.int32)


n = 23
pbirthday_naive_ans = pbirthday_naive(n)
print(
    f'{pbirthday_naive_ans*100.0:.1f}% chance of at least 1 birthday match among {n} people'
)

p = 0.50
qbirthday_naive_ans = qbirthday_naive(p)
print(
    f'For an approximately {p*100.0:.1f}% chance of at least 1 birthday match, we need {qbirthday_naive_ans} people'
)


50.7% chance of at least 1 birthday match among 23 people
For an approximately 50.0% chance of at least 1 birthday match, we need 23 people


We can also find the probability of having at least one _triple birthday match_, i.e., three people with the same birthday; with the `pbirthday` and `qbirthday` functions below, all we have to do is add `coincident=3` to say we're looking for triple matches. For example, `pbirthday(23, coincident=3)` returns 0.014, so 23 people give us only a 1.4% chance of a triple birthday match. `qbirthday(0.5, coincident=3)` returns 88, so we'd need 88 people to have at least a 50% chance of at least one triple birthday match.

In [9]:
def pbirthday(n, classes=365, coincident=2):
    """ Return the probability of a coincidence of a birthday match among n people.
    
        Python implementation of R's pbirthday {stats}.
    """
    k = coincident
    c = classes

    if k < 2:
        return 1.0
    if k == 2:
        numer = np.arange(c, c - n, -1)
        denom = np.array([c] * n)
        return 1.0 - np.sum(np.prod(numer / denom))
    if k > n:
        return 0.0
    if n > c * (k - 1):
        return 1.0
    else:
        lhs = n * np.exp(-n / (c * k)) / (1 - n / (c * (k + 1)))**(1 / k)
        lxx = k * np.log(lhs) - (k - 1) * np.log(c) - special.gammaln(k + 1)
        return -np.expm1(-np.exp(lxx))


def qbirthday(prob=0.5, classes=365, coincident=2):
    """ Return the smallest number of people needed to have at least
        the specified probability of coincidence.
    
        Python implementation of R's qbirthday {stats}.
    """
    k = coincident
    c = classes
    p = prob

    if p <= 0.0:
        return 1
    if p >= 1.0:
        return c * (k - 1) + 1

    N = np.exp((
        (k - 1) * np.log(c) + special.gammaln(k + 1) + np.log(-np.log1p(-p))) /
               k)
    N = np.ceil(N).astype(np.int32)
    if pbirthday(N, c, k) < p:
        N += 1
        while pbirthday(N, c, k) < p:
            N += 1
    elif pbirthday(N - 1, c, k) >= p:
        N -= 1
        while pbirthday(N - 1, c, k) >= p:
            N -= 1
    return N


n = 23
pbirthday_ans = pbirthday(n, coincident=3)
print(
    f'{pbirthday_ans * 100.0:.1F}% chance of triple birthday match among {n} people'
)

p = 0.5
qbirthday_ans = qbirthday(p, coincident=3)
print(
    f'We need {qbirthday_ans} people for a triple match with probability of {p * 100.0:.1F}%'
)


1.4% chance of triple birthday match among 23 people
We need 88 people for a triple match with probability of 50.0%


To simulate the birthday problem, we can use

In [10]:
np.random.seed(13)

b = np.random.choice(np.arange(1, 365+1), size=23, replace=True)
u, c = np.unique(b, return_counts=True)

to generate random birthdays for 23 people and then use [`numpy.unique`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html), specifiying `return_counts=True`,  to tally up the counts of how many people were born on each day. We can run 10<sup>4</sup> repetitions as follows:

In [11]:
np.random.seed(21)

n = 23
trials = 10**4

ordered = np.arange(1, n + 1)

m = 0
for i in range(trials):
    b = np.random.choice(np.arange(1, 365 + 1), size=n, replace=True)
    u, c = np.unique(b, return_counts=True)
    if np.sum(c > 1):
        m += 1

p = m / trials
print(
    f'In a simulation with {trials} trials, the probability of a birthday match with {n} people is {p}'
)

In a simulation with 10000 trials, the probability of a birthday match with 23 people is 0.5118


If the probabilities of various days are not all equal, the calculation becomes much more difficult, but the simulation can easily be extended since `numpy.random.choice` allows us to specify a list `p` of the probability of each day (by default sample assigns equal probabilities, so in the above the probability is 1/365 for each day).